In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np


In [3]:

img_height = 150
img_width = 150
batch_size = 32
data_directory = '/Users/baudi/AI/practicas/uvas/data/train_val/output/'
test_data_directory = '/Users/baudi/AI/practicas/uvas/data/test/output/'
val_split = 0.2
seed = 42
num_classes = 4
learning_rate = 0.00001


In [4]:

def adjustments(image):
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.9, 1.1)
    return image

def process_labels(images, labels):
    one_hot_labels = tf.one_hot(labels, depth=num_classes)
    return images, one_hot_labels

train_data_raw = image_dataset_from_directory(
    data_directory,
    validation_split=val_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

class_names = train_data_raw.class_names

train_data = train_data_raw.map(lambda x, y: (adjustments(x), y)).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)


val_data = image_dataset_from_directory(
    data_directory,
    validation_split=val_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)

test_data = image_dataset_from_directory(
    test_data_directory,
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size
).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)


Found 8000 files belonging to 4 classes.
Using 6400 files for training.
Metal device set to: Apple M2 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-04-10 11:25:53.352203: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-10 11:25:53.352362: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 8000 files belonging to 4 classes.
Using 1600 files for validation.
Found 2000 files belonging to 4 classes.


In [5]:
def build_model():

    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(512, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4 clases: sanas, enfermedad1, enfermedad2, enfermedad3
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    return model


In [6]:

def cross_validate(model_builder, n_splits=2, epochs=20):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    X = np.concatenate([x for x, y in train_data] + [x for x, y in val_data])
    y = np.concatenate([y for x, y in train_data] + [y for x, y in val_data])

    accuracies = []
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y), start=1):
        model = model_builder()
        print(f"Comienza el entrenamiento del fold {fold}")
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)

        callbacks = [early_stopping, reduce_lr]

        history = model.fit(X[train_idx],
                  y[train_idx],
                  validation_data=(X[val_idx], y[val_idx]),
                  batch_size = batch_size,
                  epochs=epochs,
                  callbacks=callbacks)
        print(f"Termina el entrenamiento del fold {fold}")
        val_accuracy = history.history['val_accuracy']
        last_val_accuracy = val_accuracy[-1]
        print(f"Precisión de validación del fold {fold}: {last_val_accuracy}")

        accuracies.append(last_val_accuracy)

    return np.mean(accuracies)


In [7]:

mean_accuracy = cross_validate(build_model,5,20)
print(f'Mean accuracy: {mean_accuracy:.4f}')


2023-04-10 11:25:53.686794: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Comienza el entrenamiento del fold 1
Epoch 1/20


2023-04-10 11:25:55.751648: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - ETA: 0s - loss: 1.0864 - accuracy: 0.6408

2023-04-10 11:26:04.598700: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2023-04-10 11:31:50.268405: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


199/200 [============================>.] - ETA: 0s - loss: 1.0325 - accuracy: 0.6573

2023-04-10 11:32:00.153444: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


200/200 [==============================] - 11s 51ms/step - loss: 1.0308 - accuracy: 0.6578 - val_loss: 0.8749 - val_accuracy: 0.6962 - lr: 1.0000e-04
Epoch 2/20
200/200 [==============================] - 9s 44ms/step - loss: 0.5673 - accuracy: 0.7903 - val_loss: 0.5317 - val_accuracy: 0.8156 - lr: 1.0000e-04
Epoch 3/20
200/200 [==============================] - 9s 45ms/step - loss: 0.4008 - accuracy: 0.8520 - val_loss: 0.3496 - val_accuracy: 0.8575 - lr: 1.0000e-04
Epoch 4/20
200/200 [==============================] - 9s 44ms/step - loss: 0.2708 - accuracy: 0.9016 - val_loss: 0.4118 - val_accuracy: 0.8531 - lr: 1.0000e-04
Epoch 5/20
200/200 [==============================] - 9s 44ms/step - loss: 0.2372 - accuracy: 0.9150 - val_loss: 0.2906 - val_accuracy: 0.8906 - lr: 1.0000e-04
Epoch 6/20
200/200 [==============================] - 9s 44ms/step - loss: 0.1780 - accuracy: 0.9367 - val_loss: 0.3035 - val_accuracy: 0.8950 - lr: 1.0000e-04
Epoch 7/20
200/200 [==============================

KeyboardInterrupt: 

In [8]:

def train_final_model(model_builder, epochs=20):
    X = np.concatenate([x for x, y in train_data] + [x for x, y in val_data])
    y = np.concatenate([y for x, y in train_data] + [y for x, y in val_data])

    final_model = model_builder()

    # Definir el callback de EarlyStopping
    early_stopping = EarlyStopping(monitor='loss', patience=3, min_delta=0.001)

    # Agregar el callback al método fit
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)

    callbacks = [early_stopping, reduce_lr]
    final_model.fit(X, y, epochs=epochs, batch_size=batch_size, callbacks=callbacks)

    return final_model

final_model = train_final_model(build_model,50)


Epoch 1/50


2023-04-10 11:34:10.527539: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 12s 42ms/step - loss: 0.9953 - accuracy: 0.6705 - lr: 1.0000e-04
Epoch 2/50
250/250 [==============================] - 10s 40ms/step - loss: 0.5207 - accuracy: 0.8030 - lr: 1.0000e-04
Epoch 3/50
250/250 [==============================] - 10s 41ms/step - loss: 0.3511 - accuracy: 0.8649 - lr: 1.0000e-04
Epoch 4/50
250/250 [==============================] - 10s 41ms/step - loss: 0.2655 - accuracy: 0.9023 - lr: 1.0000e-04
Epoch 5/50
250/250 [==============================] - 11s 44ms/step - loss: 0.1991 - accuracy: 0.9243 - lr: 1.0000e-04
Epoch 6/50
250/250 [==============================] - 10s 42ms/step - loss: 0.1533 - accuracy: 0.9459 - lr: 1.0000e-04
Epoch 7/50
250/250 [==============================] - 10s 40ms/step - loss: 0.1247 - accuracy: 0.9545 - lr: 1.0000e-04
Epoch 8/50
250/250 [==============================] - 10s 40ms/step - loss: 0.1098 - accuracy: 0.9618 - lr: 1.0000e-04
Epoch 9/50
250/250 [==============================] - 10s 4

In [ ]:
test_loss, test_accuracy = final_model.evaluate(test_data)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

Y_pred = final_model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)

y_true = np.concatenate([y.numpy() for _, y in test_data.unbatch()])
y_true_labels = np.argmax(y_true.reshape(-1, len(class_names)), axis=1)

print('Classification Report:')
print(classification_report(y_true_labels, y_pred, target_names=class_names))

print('Confusion Matrix:')
print(confusion_matrix(y_true_labels, y_pred))
